# e-Radの応募データから主要な項目を抽出して、ローカルのMariaDBに保存するプログラム

e-Radの事務代表者アカウントから自機関の応募データをCSV形式でダウンロードできるのですが、そのままだとカラムが3452個もあります。研究分担者のあたりから後ろのカラムを手作業で削除してもいいのですが、ここでは、必要なカラムだけ自動で取り出して、ローカルのMariaDBに保存することにします。

In [ ]:
import csv
import pandas as pd
import numpy as np
import glob

CSVファイルを読み込んで、主要な項目を取り出す関数です。

In [ ]:
def read_erad_csv(filename):
    with open(csvfile, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        header = next(reader)

        oubolist = []
        for row in reader:
            oubo = [
                row[1],
                row[3],
                row[5],
                row[6],
                row[7],
                row[8],
                row[9],
                row[10],
                row[11],
                row[12],
                row[13],
                row[14],
                row[18],
                row[22],
                row[45],
                row[46],
                row[47],
                row[50],
                row[51],
                row[52],
                row[53],
                row[54],
                row[55],
            ]
            oubolist.append(oubo)
    df = pd.DataFrame(oubolist)
    df.columns = [
        'kadai_id',
        'oubo_saitaku_bangou',
        'nendo',
        'seido_code',
        'seidomei',
        'jigyou_code',
        'jigyoumei',
        'koubo_group_code',
        'koubo_groupmei',
        'koubonendo',
        'koubo_code',
        'koubomei',
        'kadaimei',
        'gyoumu_main_status',
        'kenkyuusha_id',
        'shimeikanji_sei',
        'shimeikanji_mei',
        'kenkyuukikan_code',
        'kenkyuukikanmei',
        'bukyoku_code',
        'bukyokumei',
        'shokkai_code',
        'yakushoku',
    ]
    
    return df

準備：空のデータフレームを作っておきます。

In [ ]:
columns = [
    'kadai_id',
    'oubo_saitaku_bangou',
    'nendo',
    'seido_code',
    'seidomei',
    'jigyou_code',
    'jigyoumei',
    'koubo_group_code',
    'koubo_groupmei',
    'koubonendo',
    'koubo_code',
    'koubomei',
    'kadaimei',
    'gyoumu_main_status',
    'kenkyuusha_id',
    'shimeikanji_sei',
    'shimeikanji_mei',
    'kenkyuukikan_code',
    'kenkyuukikanmei',
    'bukyoku_code',
    'bukyokumei',
    'shokkai_code',
    'yakushoku',
]
df = pd.DataFrame(columns=columns)

カレントディレクトリの下に、rawdata_ouboという名前でフォルダを作って、そこに応募データのCSVファイルを置いておきます。自分は年度ごと（2006-2018）にCSVファイルを作って置いています。上記の関数を、CSVファイルの個数分ループさせて、ひとつの大きなデータフレームを作ります。

In [ ]:
for csvfile in glob.glob('rawdata_oubo/*.csv'):
    annual = read_erad_csv(filename=csvfile)
    df = pd.concat([df, annual])

df

indexをリセットしておきます。

In [ ]:
df = df.reset_index(drop=True)
df

データフレームの様子を見ておきます。

In [ ]:
df.info()

年度のデータ型をintにしておきます。

In [ ]:
df = df.astype({
    'nendo': int,
})
df.info()

ローカルのMariaDBに保存します

In [ ]:
import configparser
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

config = configparser.ConfigParser()
config.read('../settings/config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

engine = create_engine(url, echo=True)
df.to_sql('erad_oubo', engine, if_exists='replace')

コミットされたことを確認して、おしまいです。